In [2]:
from cropharvest.datasets import CropHarvest
from cropharvest.inference import Inference
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier

import requests
import tempfile

DATA_DIR = "data"

!mkdir $DATA_DIR

A subdirectory or file data already exists.


In [3]:
from sklearn.ensemble import RandomForestClassifier

from cropharvest.datasets import Task, CropHarvest
from cropharvest.countries import get_country_bbox

data_dir = "data"

Kenya_Bounding_Box = get_country_bbox("Kenya")
Kenya_Bounding_Box = Kenya_Bounding_Box[0]

In [4]:
kenya_task = Task(bounding_box=Kenya_Bounding_Box, normalize=True)
kenya_dataset = CropHarvest(data_dir, kenya_task)

In [5]:
evaluation_datasets = CropHarvest.create_benchmark_datasets(DATA_DIR)
evaluation_datasets
kenya_dataset = evaluation_datasets[0]

In [8]:
import config
import json
from cropharvest.engineer import TestInstance

sample_sizes = config.DATASET_TO_SIZES['Kenya']
# sample_sizes = [20]
seed_average = False


print(sample_sizes)
# seed = 0
# kenya_dataset.shuffle(seed=seed)

Skip_Existing_Results = True
   
for seed in config.SHUFFLE_SEEDS:
    kenya_dataset.shuffle(seed=seed)
    for sample in sample_sizes:
        print("Running sample size: ", sample)
        json_suffix = f"{kenya_dataset.id}_{sample}_{seed}.json"
        nc_suffix  = f"{kenya_dataset.id}_{sample}_{seed}.nc"
        average_suffix = f"{kenya_dataset.id}_{sample}*.nc"

        train_x, train_y = kenya_dataset.as_array(flatten_x=True, num_samples=sample)
        print(train_x.shape, train_y.shape)
        model = RandomForestClassifier(random_state=42)
        model.fit(train_x, train_y)

        for test_id, test_instance in kenya_dataset.test_data(flatten_x=True):
            results_json =  "Results/RF/Kenya/" + json_suffix
            results_nc =  "Results/RF/Kenya/" + nc_suffix

            if Path(results_json).exists() and Skip_Existing_Results:
                print(f"Results already saved for {results_json} - skipping")

            else:
                preds = model.predict_proba(test_instance.x)
                print(preds[0:10], preds.shape)
                preds = preds[:, 1]
                print(preds[0:10], preds.shape)
                results = test_instance.evaluate_predictions(preds)
                print(results)

                with Path(results_json).open("w") as f:
                    json.dump(results, f)

                ds = test_instance.to_xarray(preds)
                ds.to_netcdf(results_nc)

        # finally, we want to get results when all the test instances are considered
                    # together
    if seed_average:
        print("Running seed average")
        all_nc_files = list(Path('Results/RF/Kenya/').glob(f"*{seed}.nc"))
        print(all_nc_files)
        combined_instance, combined_preds = TestInstance.load_from_nc(all_nc_files)

        combined_results = combined_instance.evaluate_predictions(combined_preds)

        with (Path('Results/Kenya/Kenya_Combined/') / f"Seed_combined_{json_suffix}").open("w") as f:
            json.dump(combined_results, f)
if not seed_average:
    print("Averaging using sample size")
    for sample in sample_sizes:
        average_suffix = f"{kenya_dataset.id}_{sample}"
        all_nc_files = list(Path('Results/RF/Kenya/').glob(f"{average_suffix}*.nc"))
        print(all_nc_files)
        combined_instance, combined_preds = TestInstance.load_from_nc(all_nc_files)

        combined_results = combined_instance.evaluate_predictions(combined_preds)

        with (Path('Results/RF/Kenya_Combined/') / f"Sample_combined_{average_suffix}.json").open("w") as f:
            json.dump(combined_results, f)







[None]
Running sample size:  None
(6640, 216) (6640,)
[[0.59 0.41]
 [0.53 0.47]
 [0.61 0.39]
 [0.68 0.32]
 [0.73 0.27]
 [0.69 0.31]
 [0.62 0.38]
 [0.59 0.41]
 [0.64 0.36]
 [0.68 0.32]] (7600, 2)
[0.41 0.47 0.39 0.32 0.27 0.31 0.38 0.41 0.36 0.32] (7600,)
{'auc_roc': 0.8663689358209163, 'f1_score': 0.7887323943661972, 'iou': 0.6511627906976745, 'num_samples': 538}
Results already saved for Results/RF/Kenya/Kenya_1_maize_None_0.json - skipping
Running sample size:  None
(6640, 216) (6640,)
[[0.54 0.46]
 [0.51 0.49]
 [0.61 0.39]
 [0.65 0.35]
 [0.62 0.38]
 [0.66 0.34]
 [0.51 0.49]
 [0.45 0.55]
 [0.6  0.4 ]
 [0.62 0.38]] (7600, 2)
[0.46 0.49 0.39 0.35 0.38 0.34 0.49 0.55 0.4  0.38] (7600,)
{'auc_roc': 0.8279769606410952, 'f1_score': 0.7561837455830389, 'iou': 0.6079545454545454, 'num_samples': 538}
Results already saved for Results/RF/Kenya/Kenya_1_maize_None_1.json - skipping
Running sample size:  None
(6640, 216) (6640,)
[[0.64 0.36]
 [0.55 0.45]
 [0.61 0.39]
 [0.67 0.33]
 [0.66 0.34]
 [0